[INDEX](Kubernetes.ipynb)  [NEXT](3.pods.ipynb)

<img align="left" src="https://avatars1.githubusercontent.com/u/13629408?s=400&v=4" width="60" />

## 3. Running Pods, Deployments

<pre style="align:center" >
For this section of the lab we will continue to work with our N worker-node cluster.
</pre>
<img src="images/KubeNodes_1m_2w.svg.png" width="400" />
    
<br/><table align="right" style='font-family:"Courier New", Courier, monospace; font-size:100%; bottom:0px;'>
    <tr><td bgcolor="#ffffff" color="#ffffff"></td><td bgcolor="#ffffff" color="#ffffff"> <img src="http://www.icon100.com/up/3011/128/Twitter-alt.png" width="40"> </td><td bgcolor="#ffffff" color="#ffffff" > @mjbright </td></tr></table>

# 3. Running Pods, Deployments

<pre style="align:center" >

**In this section** we will see how we can launch pods directly from the command-line using 'kubectl run' and how it creates several resources.

This is useful for experimentation but it is not declarative.
We will also see how we can create resources from a yaml file using the 'kubectl create' command, and how we can create several containers in a Pod.

We will look at how to scale Pods, and what happens when a Pod fails
</pre>
    
<br/><table align="right" style='font-family:"Courier New", Courier, monospace; font-size:100%; bottom:0px;'>
    <tr><td bgcolor="#ffffff" color="#ffffff"></td><td bgcolor="#ffffff" color="#ffffff"> <img src="http://www.icon100.com/up/3011/128/Twitter-alt.png" width="40"> </td><td bgcolor="#ffffff" color="#ffffff" > @mjbright </td></tr></table>

    
# Kubernetes Pods

## Useful functions

Here we define some useful functions to be able to run this notebook in isolation:

In [2]:
kubectl get all

NAME                 TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
service/kubernetes   ClusterIP   10.0.0.1     <none>        443/TCP   15h


## Deploying Pods via 'kubectl run'

<pre style="align:center" >
We've seen how we can interrogate our cluster server using the kubectl tool, obtaining information about Resources such as Pods, Nodes in the system.

Now let's create our own pods using 'kubectl run'.  We will see that this actually creates a Deployment and a ReplicaSet, as the example below:

<img src="images/KubernetesArchi_Deploy.svg.png" width="600" />

<br/><table align="right" style='font-family:"Courier New", Courier, monospace; font-size:100%; bottom:0px;'>
    <tr><td bgcolor="#ffffff" color="#ffffff"></td><td bgcolor="#ffffff" color="#ffffff"> <img src="http://www.icon100.com/up/3011/128/Twitter-alt.png" width="40"> </td><td bgcolor="#ffffff" color="#ffffff" > @mjbright </td></tr></table>

Normally we would not create pods directly using command-line arguments, as we will do here.

We would create objects in a repeatable, declarative way from yaml files, but we will do that later.

The below ```kubectl run``` command will create 2 pods (replicas), each running just 1 container from the k8s-demo image.

Note that we can specify labels on any kubernetes objects, using the -l option followd by key=value pairs, e.g.
    -l "key1=val1,key2=val2"
     
These labels are extremely useful allowing us to select a group of objects using a label selector.

Labels might be used to mark components of the cluster as dev, test or production for example.

Or they might be used to signal hardware specifics of a node, to distinguish physical nodes with SSD disks, or GPU which could then be prioritized for particular types of processing which might be disk i/o or CPU intensive.

k8sdemo is a small Docker image of 6MBy on the Docker hub [HERE](https://hub.docker.com/r/mjbright/k8s-demo/tags/)

<br/><table align="right" style='font-family:"Courier New", Courier, monospace; font-size:100%; bottom:0px;'>
    <tr><td bgcolor="#ffffff" color="#ffffff"></td><td bgcolor="#ffffff" color="#ffffff"> <img src="http://www.icon100.com/up/3011/128/Twitter-alt.png" width="40"> </td><td bgcolor="#ffffff" color="#ffffff" > @mjbright </td></tr></table>


We want to see what Resources are created, so let's just check again what exists already ... by running 'kubectl get all'


In [3]:
kubectl get all

NAME                 TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
service/kubernetes   ClusterIP   10.0.0.1     <none>        443/TCP   15h


Next we will run 'kubectl run' and immediately run 'kubectl get all' again.

In [4]:
kubectl run k8s-demo -l "app=k8s-demo" --image=mjbright/k8s-demo:1 --replicas=2 --port=8080
kubectl get all

deployment.apps "k8s-demo" created
NAME                            READY     STATUS              RESTARTS   AGE
pod/k8s-demo-6b76bd84d7-47b2q   0/1       ContainerCreating   0          3s
pod/k8s-demo-6b76bd84d7-6ss2r   0/1       ContainerCreating   0          3s

NAME                 TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
service/kubernetes   ClusterIP   10.0.0.1     <none>        443/TCP   15h

NAME                             DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
deployment.extensions/k8s-demo   2         2         2            0           3s

NAME                                        DESIRED   CURRENT   READY     AGE
replicaset.extensions/k8s-demo-6b76bd84d7   2         2         0         3s

NAME                       DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
deployment.apps/k8s-demo   2         2         2            0           3s

NAME                                  DESIRED   CURRENT   READY     AGE
replicaset.apps/k8s-demo-6b76bd84d7   2   

In the above output we see that the only running resource initially was an internal service.

Once we perform '''kubectl run'' this spawns a **deployment** called *k8s-demo*, which creates a corrsponding **replicaset** which in turn spawns 2 **pods**.

As we performed the '''kubectl get all''' immediately after the run, we see the status before the pod containers have been created ... in fact there is a delay whilst the images are downloaded (for a first run) and pods instantiated.

Re-running '''kubectl get all''' shows the pods as *running* now.

In [5]:
kubectl get all

NAME                            READY     STATUS    RESTARTS   AGE
pod/k8s-demo-6b76bd84d7-47b2q   1/1       Running   0          50s
pod/k8s-demo-6b76bd84d7-6ss2r   1/1       Running   0          50s

NAME                 TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
service/kubernetes   ClusterIP   10.0.0.1     <none>        443/TCP   15h

NAME                             DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
deployment.extensions/k8s-demo   2         2         2            2           50s

NAME                                        DESIRED   CURRENT   READY     AGE
replicaset.extensions/k8s-demo-6b76bd84d7   2         2         2         50s

NAME                       DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
deployment.apps/k8s-demo   2         2         2            2           50s

NAME                                  DESIRED   CURRENT   READY     AGE
replicaset.apps/k8s-demo-6b76bd84d7   2         2         2         50s


... with more information, using option '*-o wide*'

In [6]:
kubectl get all -o wide

NAME                            READY     STATUS    RESTARTS   AGE       IP           NODE
pod/k8s-demo-6b76bd84d7-47b2q   1/1       Running   0          57s       10.244.8.5   aks-nodepool1-35637019-1
pod/k8s-demo-6b76bd84d7-6ss2r   1/1       Running   0          57s       10.244.4.5   aks-nodepool1-35637019-9

NAME                 TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE       SELECTOR
service/kubernetes   ClusterIP   10.0.0.1     <none>        443/TCP   15h       <none>

NAME                             DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE       CONTAINERS   IMAGES                SELECTOR
deployment.extensions/k8s-demo   2         2         2            2           57s       k8s-demo     mjbright/k8s-demo:1   app=k8s-demo

NAME                                        DESIRED   CURRENT   READY     AGE       CONTAINERS   IMAGES                SELECTOR
replicaset.extensions/k8s-demo-6b76bd84d7   2         2         2         57s       k8s-demo     mjbright/k8s-de


Note in the above output we can see that our k8s-demo pods are running on kube-node-1 and kube-node-2.

## Creating resources using yaml specifications

### Obtaining yaml as output

Using the option '-o yaml' to any 'kubectl get' command we can obtain a yaml definition, so instead of

In [7]:
kubectl get deploy/k8s-demo -o wide

NAME       DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE       CONTAINERS   IMAGES                SELECTOR
k8s-demo   2         2         2            2           1m        k8s-demo     mjbright/k8s-demo:1   app=k8s-demo


we get ...

In [8]:
kubectl get deploy/k8s-demo -o yaml

apiVersion: extensions/v1beta1
kind: Deployment
metadata:
  annotations:
    deployment.kubernetes.io/revision: "1"
  creationTimestamp: 2018-10-06T09:52:45Z
  generation: 1
  labels:
    app: k8s-demo
  name: k8s-demo
  namespace: default
  resourceVersion: "115387"
  selfLink: /apis/extensions/v1beta1/namespaces/default/deployments/k8s-demo
  uid: 934c6208-c94d-11e8-a432-6ad902cee12c
spec:
  replicas: 2
  selector:
    matchLabels:
      app: k8s-demo
  strategy:
    rollingUpdate:
      maxSurge: 1
      maxUnavailable: 1
    type: RollingUpdate
  template:
    metadata:
      creationTimestamp: null
      labels:
        app: k8s-demo
    spec:
      containers:
      - image: mjbright/k8s-demo:1
        imagePullPolicy: IfNotPresent
        name: k8s-demo
        ports:
        - containerPort: 8080
          protocol: TCP
        resources: {}
        terminationMessagePath: /dev/termination-log
        terminationMessagePolicy: File
      dnsPolicy: ClusterFirst
      restartPol

## Creating resources from a yaml file

This is the usual way in which we would create Resources on our cluster as it is declarative and repeatable.

## Obtaining the example yaml

If you are using one of the VMS created for this lab then the example code will have been placed under:
~/src/git/ContainerOrchestration.Labs/Orchestration-Kubernetes/

If you are running in your own environment, or under play-with-kubernetes.com, you will first need to clone the repository:

```git clone https://github.com/ContainerOrchestration/Labs```

**NOTE**: You may also need to checkout the current branch, e.g.:FOSSAsia2018

```git checkout FOSSAsia2018```

## Inspecting the example

In the example yaml below, we see that we specify a Pod to be created (this is the smallest unit of execution in a Kubernetes cluster) which will contains 2 containers.

The yaml specifies which Kubernetes api version to use, the "*kind*" of object to be managed, and it's name.

In [9]:
cat examples/3.pod_example.yaml


apiVersion: v1
kind: Pod
metadata:
  name: 2-containers-in-a-pod
spec:
  containers:
  - name: k8s-demo
    image: mjbright/k8s-demo:1
    ports:
    - containerPort: 9876
  - name: shell
    image: alpine
    command:
      - "bin/sh"
      - "-c"
      - "while true; do date; sleep 1; done"



Now we can create our pod as follows:

In [11]:
kubectl create -f examples/3.pod_example.yaml

pod "2-containers-in-a-pod" created


In [16]:
kubectl get po/2-containers-in-a-pod

NAME                    READY     STATUS              RESTARTS   AGE
2-containers-in-a-pod   0/2       ContainerCreating   0          0s


In [12]:
kubectl get po/2-containers-in-a-pod

NAME                    READY     STATUS    RESTARTS   AGE
2-containers-in-a-pod   2/2       Running   0          15s


In [13]:
kubectl describe po/2-containers-in-a-pod

Name:         2-containers-in-a-pod
Namespace:    default
Node:         aks-nodepool1-35637019-6/10.240.0.12
Start Time:   Sat, 06 Oct 2018 11:55:02 +0200
Labels:       <none>
Annotations:  <none>
Status:       Running
IP:           10.244.6.5
Containers:
  k8s-demo:
    Container ID:   docker://0b97523dd301794c807a8ae8a4d61876e8d0e9d4f0e81041053e0ed695d21e44
    Image:          mjbright/k8s-demo:1
    Image ID:       docker-pullable://mjbright/k8s-demo@sha256:6d84d697796e0a529028808ba76e92482c4ca0c408e9f9eae8aba6f23c173264
    Port:           9876/TCP
    Host Port:      0/TCP
    State:          Running
      Started:      Sat, 06 Oct 2018 11:55:07 +0200
    Ready:          True
    Restart Count:  0
    Environment:    <none>
    Mounts:
      /var/run/secrets/kubernetes.io/serviceaccount from default-token-mk5wp (ro)
  shell:
    Container ID:  docker://e5308a54edac8a798e35bde7c77b2f27f85439c75706e15fdffb73adb035254b
    Image:         alpine
    Image ID:      docker-pullable://al

In [19]:
kubectl exec -it 2-containers-in-a-pod -n default echo hello world

Defaulting container name to k8s-demo.
Use 'kubectl describe pod/2-containers-in-a-pod -n default' to see all of the containers in this pod.
hello world


In [14]:
kubectl logs 2-containers-in-a-pod shell | head -3

Sat Oct  6 09:55:10 UTC 2018
Sat Oct  6 09:55:11 UTC 2018
Sat Oct  6 09:55:12 UTC 2018


It would be interesting to get the sidecar container to update the main k8s-demo container output (kubernetes icon in text format).  This could be done by changing the exec arguments in the yaml file used to create the deployment ... next time ;-)

In [15]:
kubectl delete po/2-containers-in-a-pod

pod "2-containers-in-a-pod" deleted


In [16]:
kubectl get pods

NAME                        READY     STATUS        RESTARTS   AGE
2-containers-in-a-pod       2/2       Terminating   0          7m
k8s-demo-6b76bd84d7-47b2q   1/1       Running       0          9m
k8s-demo-6b76bd84d7-6ss2r   1/1       Running       0          9m


## What happens if a pod dies?

In [17]:
kubectl get pods -o wide

NAME                        READY     STATUS        RESTARTS   AGE       IP           NODE
2-containers-in-a-pod       2/2       Terminating   0          7m        10.244.6.5   aks-nodepool1-35637019-6
k8s-demo-6b76bd84d7-47b2q   1/1       Running       0          9m        10.244.8.5   aks-nodepool1-35637019-1
k8s-demo-6b76bd84d7-6ss2r   1/1       Running       0          9m        10.244.4.5   aks-nodepool1-35637019-9


When we deleted the 2-containers-in-a-pod Pod it just died.

If we delete one of our k8s-demo Pods we will see that a new Pod is created automatically, this is because it is managed by a ReplicaSet.

Running a Pod via "kubectl run" creates a Deployment which then creates a ReplicaSet.

Let's kill one of our Pods and see what happens


In [24]:
kubectl get pod | grep k8s-demo | head -1 | awk '{ print $1;}'

POD1=$(kubectl get pod | grep k8s-demo | head -1 | awk '{ print $1;}')

k8s-demo-6b76bd84d7-6ss2r


In [25]:
kubectl delete pod $POD1

pod "k8s-demo-6b76bd84d7-6ss2r" deleted


In [26]:
kubectl get pods -o wide

NAME                        READY     STATUS              RESTARTS   AGE       IP           NODE
k8s-demo-6b76bd84d7-6ss2r   0/1       Terminating         0          11m       10.244.4.5   aks-nodepool1-35637019-9
k8s-demo-6b76bd84d7-v4v2c   1/1       Running             0          1m        10.244.3.5   aks-nodepool1-35637019-5
k8s-demo-6b76bd84d7-vsnj9   0/1       ContainerCreating   0          5s        <none>       aks-nodepool1-35637019-7


In [27]:
sleep 10; kubectl get pods -o wide

NAME                        READY     STATUS    RESTARTS   AGE       IP           NODE
k8s-demo-6b76bd84d7-v4v2c   1/1       Running   0          2m        10.244.3.5   aks-nodepool1-35637019-5
k8s-demo-6b76bd84d7-vsnj9   1/1       Running   0          19s       10.244.9.4   aks-nodepool1-35637019-7


We see that our Pod was deleted but that a new one was created, this is what we expect when our Pod is managed by a ReplicaSet.

# Labels and Selectors

Labels can be applied to any Kubernetes generated objects and this can be useful to be able to apply operations to only selected objects based on their labels.

We can see that our k8s-demo Deployment earlier had labels automatically assigned to it.

We can create other Deployments, or Pods with a specific label.

In [28]:
kubectl describe deploy k8s-demo #| grep Labels:

Name:                   k8s-demo
Namespace:              default
CreationTimestamp:      Sat, 06 Oct 2018 11:52:45 +0200
Labels:                 app=k8s-demo
Annotations:            deployment.kubernetes.io/revision=1
Selector:               app=k8s-demo
Replicas:               2 desired | 2 updated | 2 total | 2 available | 0 unavailable
StrategyType:           RollingUpdate
MinReadySeconds:        0
RollingUpdateStrategy:  1 max unavailable, 1 max surge
Pod Template:
  Labels:  app=k8s-demo
  Containers:
   k8s-demo:
    Image:        mjbright/k8s-demo:1
    Port:         8080/TCP
    Host Port:    0/TCP
    Environment:  <none>
    Mounts:       <none>
  Volumes:        <none>
Conditions:
  Type           Status  Reason
  ----           ------  ------
  Available      True    MinimumReplicasAvailable
OldReplicaSets:  <none>
NewReplicaSet:   k8s-demo-6b76bd84d7 (2/2 replicas created)
Events:
  Type    Reason             Age   From                   Message
  ----    ------           

Now let's create a 2nd deployment with a label we assign:

In [29]:
kubectl run k8s-demo-test -l "mylabel=testing-v3" --image=mjbright/k8s-demo:3

deployment.apps "k8s-demo-test" created


In [30]:
kubectl get po | grep k8s-demo

k8s-demo-6b76bd84d7-v4v2c       1/1       Running             0          2m
k8s-demo-6b76bd84d7-vsnj9       1/1       Running             0          39s
k8s-demo-test-c5d55945b-gdvz2   0/1       ContainerCreating   0          4s


Now we can choose to operate on, e.g. delete, only objects matching a label - note we must delete the Deployment itself no the Pods else they will be recreated.

In [31]:
kubectl delete deploy -l "mylabel=testing-v3"

deployment.extensions "k8s-demo-test" deleted


In [32]:
kubectl get po | grep k8s-demo

k8s-demo-6b76bd84d7-v4v2c       1/1       Running       0          2m
k8s-demo-6b76bd84d7-vsnj9       1/1       Running       0          1m
k8s-demo-test-c5d55945b-gdvz2   0/1       Terminating   0          25s
You have new mail in /var/mail/mjb


This is an extremely important concept in Kubernetes allowing a loose coupling of elements, where Controllers can keep track of elements of the cluster and request resources based on their labels.

## Changing a label

We can also try removing a label from a Pod which was assigned by it's Controller (ReplicaSet).

In [34]:
kubectl get pods --show-labels

NAME                        READY     STATUS    RESTARTS   AGE       LABELS
k8s-demo-6b76bd84d7-v4v2c   1/1       Running   0          6m        app=k8s-demo,pod-template-hash=2632684083
k8s-demo-6b76bd84d7-vsnj9   1/1       Running   0          4m        app=k8s-demo,pod-template-hash=2632684083


Let's use the

```kubectl label pod <pod> app-```

command to remove the app label which the ReplicaSet uses:
    

In [37]:
kubectl label pod $(kubectl get pods --no-headers | awk '{ print $1; exit(0); }') app-

pod "k8s-demo-6b76bd84d7-v4v2c" labeled


## Can you explain what happened? - see below

In [38]:
kubectl get pods --show-labels

NAME                        READY     STATUS    RESTARTS   AGE       LABELS
k8s-demo-6b76bd84d7-qmzrl   1/1       Running   0          15s       app=k8s-demo,pod-template-hash=2632684083
k8s-demo-6b76bd84d7-v4v2c   1/1       Running   0          10m       pod-template-hash=2632684083
k8s-demo-6b76bd84d7-vsnj9   1/1       Running   0          8m        app=k8s-demo,pod-template-hash=2632684083


What happened is that the replicaSet created Pods with label app=k8s-demo and it uses this to know which Pods to manage.  As we removed the label, that Pod was no longer managed by the ReplicaSet, which then launched a new instance so that it manages the same 2 replicas.

Now let's delete the unmanaged instance before proceeding

In [46]:
kubectl delete pod $(kubectl get pods --show-labels | grep k8s-demo | grep -v app= | awk '{ print $1; }')

pod "k8s-demo-6b76bd84d7-v4v2c" deleted


# Scaling our application

As our Pods are managed by a **ReplicaSet** we can scale the number of replicas by using the scale command on the **Deployment**:

In [47]:
kubectl scale --replicas=3 deploy/k8s-demo

deployment.extensions "k8s-demo" scaled


In [48]:
kubectl get all

NAME                            READY     STATUS    RESTARTS   AGE
pod/k8s-demo-6b76bd84d7-nt4bd   1/1       Running   0          5s
pod/k8s-demo-6b76bd84d7-qmzrl   1/1       Running   0          4m
pod/k8s-demo-6b76bd84d7-vsnj9   1/1       Running   0          12m

NAME                 TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
service/kubernetes   ClusterIP   10.0.0.1     <none>        443/TCP   16h

NAME                             DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
deployment.extensions/k8s-demo   3         3         3            3           24m

NAME                                        DESIRED   CURRENT   READY     AGE
replicaset.extensions/k8s-demo-6b76bd84d7   3         3         3         24m

NAME                       DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
deployment.apps/k8s-demo   3         3         3            3           24m

NAME                                  DESIRED   CURRENT   READY     AGE
replicaset.apps/k8s-demo-6b76bd84d7  

In [49]:
kubectl get pods -o wide

NAME                        READY     STATUS    RESTARTS   AGE       IP           NODE
k8s-demo-6b76bd84d7-nt4bd   1/1       Running   0          8s        10.244.4.6   aks-nodepool1-35637019-9
k8s-demo-6b76bd84d7-qmzrl   1/1       Running   0          4m        10.244.6.7   aks-nodepool1-35637019-6
k8s-demo-6b76bd84d7-vsnj9   1/1       Running   0          12m       10.244.9.4   aks-nodepool1-35637019-7


### Scaling down

We can of course scale back down:

In [50]:
kubectl scale --replicas=2 deploy/k8s-demo

deployment.extensions "k8s-demo" scaled


In [51]:
kubectl get pods -o wide

NAME                        READY     STATUS        RESTARTS   AGE       IP           NODE
k8s-demo-6b76bd84d7-nt4bd   0/1       Terminating   0          31s       10.244.4.6   aks-nodepool1-35637019-9
k8s-demo-6b76bd84d7-qmzrl   1/1       Running       0          5m        10.244.6.7   aks-nodepool1-35637019-6
k8s-demo-6b76bd84d7-vsnj9   1/1       Running       0          13m       10.244.9.4   aks-nodepool1-35637019-7


## Accessing our application

To access our application pods we should expose them as a service.
This would have the advantage of being able to address a virtual ip address for our service as pods themselves come and go (so we don't need to determine their address), and also allows load balancing between pods.

### Accessing our application - directly accessing Pods

But there are other more basic ways that we will investigate first.

If we perform '''kubectl get pods -o wide''' we will see the cluster ip of our pods, enabling us to access directly to the pods.

In [52]:
kubectl get pods -o wide 

NAME                        READY     STATUS    RESTARTS   AGE       IP           NODE
k8s-demo-6b76bd84d7-qmzrl   1/1       Running   0          5m        10.244.6.7   aks-nodepool1-35637019-6
k8s-demo-6b76bd84d7-vsnj9   1/1       Running   0          13m       10.244.9.4   aks-nodepool1-35637019-7


In [53]:
POD1=$(kubectl get pod -o wide --no-headers | grep k8s-demo | head -1 | awk '{print $1;}')
POD2=$(kubectl get pod -o wide --no-headers | grep k8s-demo | tail -1 | awk '{print $1;}')
POD1_IP=$(kubectl get pod -o wide --no-headers | grep k8s-demo | head -1 | awk '{print $6;}')
POD2_IP=$(kubectl get pod -o wide --no-headers | grep k8s-demo | tail -1 | awk '{print $6;}')

echo POD1=$POD1 POD1_IP=$POD1_IP
echo POD2=$POD2 POD2_IP=$POD2_IP

POD1=k8s-demo-6b76bd84d7-qmzrl POD1_IP=10.244.6.7
POD2=k8s-demo-6b76bd84d7-vsnj9 POD2_IP=10.244.9.4


Let's look at the yaml information for POD1:

In [54]:
kubectl get pod $POD1 -o yaml

apiVersion: v1
kind: Pod
metadata:
  creationTimestamp: 2018-10-06T10:12:35Z
  generateName: k8s-demo-6b76bd84d7-
  labels:
    app: k8s-demo
    pod-template-hash: "2632684083"
  name: k8s-demo-6b76bd84d7-qmzrl
  namespace: default
  ownerReferences:
  - apiVersion: extensions/v1beta1
    blockOwnerDeletion: true
    controller: true
    kind: ReplicaSet
    name: k8s-demo-6b76bd84d7
    uid: 93552f2a-c94d-11e8-a432-6ad902cee12c
  resourceVersion: "117847"
  selfLink: /api/v1/namespaces/default/pods/k8s-demo-6b76bd84d7-qmzrl
  uid: 5880d2b7-c950-11e8-a432-6ad902cee12c
spec:
  containers:
  - image: mjbright/k8s-demo:1
    imagePullPolicy: IfNotPresent
    name: k8s-demo
    ports:
    - containerPort: 8080
      protocol: TCP
    resources: {}
    terminationMessagePath: /dev/termination-log
    terminationMessagePolicy: File
    volumeMounts:
    - mountPath: /var/run/secrets/kubernetes.io/serviceaccount
      name: default-token-mk5wp
      readOnly: true
  dnsPolicy: ClusterFirst
 

We can see that the containerPort 8080/TCP is exposed (as we specified on our initial '*kubectl run*' command).

So now we can access to the pods directly on their port 8080.

## To be updated (we cannot directly access pod IPs on AKS Cluster)

In [55]:
curl http://$POD1_IP:8080

In [43]:
curl http://$POD2_IP:8080




                               
                    (((((((((                  
               .(((((((((((((((((.             
           .((((((((((((&((((((((((((.         
       /((((((((((((((((@((((((((((((((((/     
      ((((((((((((((((((@((((((((((((((((((    
     *(((((##((((((@@@@@@@@@@@((((((%#(((((*   
     (((((((@@@(@@@@#((@@@((#@@@@(@@@(((((((   
    *(((((((((@@@@(((((@@@(((((@@@@(((((((((,  
    (((((((((@@@%@@@@((@@@((@@@@%@@@(((((((((  
   .(((((((((@@((((@@@@@@@@@@@((((@@(((((((((. 
   (((((((((&@@(((((@@@(((@@@(((((@@&((((((((( 
   (((((((((&@@@@@@@@@@#(#@@@@@@@@@@&((((((((( 
  ((((((@@@@@@@@(((((@@@@@@@(((((&@@@@@@@((((((
  (((((((((((%@@((((%@@@(@@@%((((@@&(((((((((((
   ((((((((((((@@@((@@%(((%@@((@@@(((((((((((( 
     (((((((((((#@@@@%(((((&@@@@#(((((((((((   
      /(((((((((((@@@@@@@@@@@@@(((((((((((/    
        (((((((((@@(((((((((((@@(((((((((      
          (((((((&(((((((((((((&(((((((        
           /(((((((((((((((((((((((((/         
     

**Note:** In a real deployment of course the actual Pod ips would not be externally accessible so we would want some way of making the nodes available on a well known address.